In [2]:
import json
import numpy as np
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from glob import glob
from gensim import matutils, models
import scipy.sparse
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus

In [9]:
with open('/home/nick/Documents/Reddit_NLP/pickles/worldnews.pickle', 'rb') as file:
    askreddit = pickle.load(file)

In [10]:
def clean(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', ' ', text)
    return text

def pre_clean2(documents):
    cleaned_list = []
    for doc in documents:
        cleaned_list.append(clean(doc))
    return cleaned_list

In [11]:
def lemmatize(text):
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    nlp.max_length = 14082871
    lemmed = []
    for comment in text:
        teckst = nlp(comment)
        lemmed.append(' '.join([word.lemma_ for word in teckst if not word.is_stop]))
    return lemmed

In [12]:
askreddit_clean = pre_clean2(askreddit)
askreddit_clean

['so was there any posts left after the purge',
 'ok just for some context  facebook has an estimated  million fake user accounts every month about  billion messages are posted between users thats  billion per week billion  nobody gives a shit about the  million that some pr intern dug up when he reviewed some bots statistics as a great message to hype the tanking facebook popularity',
 'they still havent taken down my post about mars being as big as the moon s',
 'why does facebook have a fake news problem  because facebook is not a news site its social media its a place where anybody can express their opinion no matter how accurate or misled they are  you should not take anything posted on facebook as fact without verifying it though an actual news source  stop expecting a social media site to be a news policing site',
 'a lot of karens are going to be be saying they were hacked today',
 'i bought a mask that arrived in a box that had a warning on it saying it wasnt for avoiding coro

In [13]:
lemd = lemmatize(askreddit_clean)
lemd

['post leave purge',
 'ok context   facebook estimate   million fake user account month   billion message post user s   billion week billion   give shit   million pr intern dig review bot statistic great message hype tank facebook popularity',
 'not take post mars big moon s',
 'facebook fake news problem   facebook news site social medium place anybody express opinion matter accurate mislead   post facebook fact verify actual news source   stop expect social medium site news policing site',
 'lot karen go say hack today',
 'buy mask arrive box warning say not avoid corona virus post pic warning box facebook pull spread misinformation',
 'good tldr   reduce   be bot    facebook inc say tuesday remove   million post second quarter share false information novel coronavirus include content promote fake preventative measure exaggerated cure      big social medium company remove   million post contain hate speech flagship app second quarter   million quarter    delete   million post connect

In [19]:
with open('/home/nick/Documents/Reddit_NLP/worldnews_lemmatized.pickle', 'rb') as file:
    wn = pickle.load(file)
with open('/home/nick/Documents/Reddit_NLP/politics_lemmatized.pickle', 'rb') as file:
    pt = pickle.load(file)

In [20]:
list1 = ['this', 'is like', 'a list', 'n sheeit']
list2 = ['dis', 'b', 'anoda', 'lisd']

wnpt = [wn] + [pt]
wnpt

[['post leave purge',
  'ok context   facebook estimate   million fake user account month   billion message post user s   billion week billion   give shit   million pr intern dig review bot statistic great message hype tank facebook popularity',
  'not take post mars big moon s',
  'facebook fake news problem   facebook news site social medium place anybody express opinion matter accurate mislead   post facebook fact verify actual news source   stop expect social medium site news policing site',
  'lot karen go say hack today',
  'buy mask arrive box warning say not avoid corona virus post pic warning box facebook pull spread misinformation',
  'good tldr   reduce   be bot    facebook inc say tuesday remove   million post second quarter share false information novel coronavirus include content promote fake preventative measure exaggerated cure      big social medium company remove   million post contain hate speech flagship app second quarter   million quarter    delete   million post 

In [41]:
import gensim


In [44]:
model = Word2Vec(wnpt)

INFO - 21:57:15: collecting all words and their counts
INFO - 21:57:15: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO - 21:57:15: collected 51393 word types from a corpus of 54086 raw words and 2 sentences
INFO - 21:57:15: Loading a fresh vocabulary
INFO - 21:57:15: effective_min_count=5 retains 87 unique words (0% of original 51393, drops 51306)
INFO - 21:57:15: effective_min_count=5 leaves 2078 word corpus (3% of original 54086, drops 52008)
INFO - 21:57:15: deleting the raw counts dictionary of 51393 items
INFO - 21:57:15: sample=0.001 downsamples 67 most-common words
INFO - 21:57:15: downsampling leaves estimated 596 word corpus (28.7% of prior 2078)
INFO - 21:57:15: estimated required memory for 87 words and 100 dimensions: 113100 bytes
INFO - 21:57:15: resetting layer weights
INFO - 21:57:15: training model with 3 workers on 87 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
INFO - 21:57:15: worker thread finished; awaiting f

In [49]:
model.most_similar('a')


<ipython-input-49-a15513eab837>:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  model.most_similar('a')


KeyError: "word 'a' not in vocabulary"

In [1]:
vectorizer = TfidfVectorizer()

doc_word = vectorizer.fit_transform(lemd)
words = list(np.asarray(vectorizer.get_feature_names())) 
idf_df = pd.DataFrame(doc_word.toarray(), columns=vec.get_feature_names())
idf_df

NameError: name 'TfidfVectorizer' is not defined

In [27]:
# Create the gensim corpus
corpusn = matutils.Sparse2Corpus(scipy.sparse.csr_matrix(idf_df))

# Create the vocabulary dictionary
id2wordn = dict((v, k) for k, v in vectorizer.vocabulary_.items())
word2id = dict((k, v) for k, v in vectorizer.vocabulary_.items())

In [28]:
ldan = models.LdaModel(corpus=corpusn, num_topics=13, id2word=id2wordn, passes=50)
ldan.print_topics()

INFO - 20:47:37: using symmetric alpha at 0.05
INFO - 20:47:37: using symmetric eta at 0.05
INFO - 20:47:37: using serial LDA version on this node
INFO - 20:47:37: running online (multi-pass) LDA training, 20 topics, 50 passes over the supplied corpus of 46045 documents, updating model once every 2000 documents, evaluating perplexity every 20000 documents, iterating 50x with a convergence threshold of 0.001000
INFO - 20:47:37: PROGRESS: pass 0, at document #2000/46045


IndexError: index 50527 is out of bounds for axis 1 with size 46045